## Luật Số Lớn

**(?)** Thử bỏ sung thêm về xác suất $75\%$ và $97\%$ khi tắc giả Aurélien Géron cho một analog về lý do tại sao ensemble learning của những weak learners được trở nên strong.

**(R)** Neu minh co mot dong xu voi xac xuat ra Head la $51\%$, va minh nen no $1,000$ lan, assuming ket qua independent, thi xac xuat minh duoc majority of Heads la
$$
  \sum_{k=\lceil \frac{n}{2} \rceil}^{n} (0.51)^{k} (0.49)^{n-k} \begin{pmatrix} n \\ k \end{pmatrix},
$$
where $n$ la `# tosses`.

In [2]:
import math
type(math.ceil(3.2))

int

In [11]:
from functools import reduce
reduce(lambda x, y: x*y, range(5, 8))

210

In [23]:
import math

proba_head = 0.51
proba_tail = 1 - proba_head

def my_comb(n, k):
    assert n >= k, "n must be >= k"
    mul = lambda x, y: x*y
    denominator = reduce(mul, range(1, k+1))
    numerator   = reduce(mul, range(n-(k-1), n+1))
    return numerator // denominator 
    
def proba_majority_head_after(n_tosses):
    majority = n_tosses // 2
    if n_tosses % 2 == 0:
        majority += 1
    return sum(
        #math.comb(n_tosses, k) * proba_head**k * proba_tail**(n_tosses - k)
        my_comb(n_tosses, k) * proba_head**k * proba_tail**(n_tosses - k)
        for k in range(majority, n_tosses+1))

In [24]:
proba_majority_head_after(1_000)

0.7260985557305037

Tot, nhu tac gia Aurelien Geron noi, xac xuat nay gan voi $75\%$.
Minh chay thu xem voi `10_000` tosses, co phai xac xuat se toi co $97\%$ khong.

In [38]:
proba_majority_head_after(10_000)

OverflowError: int too large to convert to float

In [37]:
try:
    proba_majority_head_after(10_000)
except OverflowError as e:
    #print(dir(e))
    print(f"OverflowError: {e}")

OverflowError: int too large to convert to float


In [43]:
from decimal import Decimal
    
def proba_majority_head_after(n_tosses):
    majority = n_tosses // 2
    if n_tosses % 2 == 0:
        majority += 1
    return sum(
        #math.comb(n_tosses, k) * proba_head**k * proba_tail**(n_tosses - k)
        Decimal(my_comb(n_tosses, k)) * Decimal(proba_head**k) * Decimal(proba_tail**(n_tosses - k))
        for k in range(majority, n_tosses+1))

In [44]:
proba_majority_head_after(10_000)

Decimal('0E-1074')

In [46]:
my_comb(10_000, 5_000) * proba_head**5_001

OverflowError: int too large to convert to float

In [47]:
float(my_comb(10_000, 5_000))

OverflowError: int too large to convert to float

In [48]:
import numpy as np

In [54]:
def binomial_term(n, k, a, b):
    mul = lambda x, y: x*y
    A = reduce(mul, a * np.arange(n-(k-1), n+1))
    B = reduce(mul, b / np.arange(1, k+1))
    return A * B

def proba_majorite_face_apres(n_tosses):
    majority = n_tosses // 2
    if n_tosses % 2 == 0:
        majority += 1
    return sum(binomial_term(n_tosses, k, proba_head, proba_tail)
               for k in range(majority, n_tosses+1))

In [55]:
proba_majorite_face_apres(10_000)

/home/phunc20/.local/bin/miniconda3/envs/homl1/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/phunc20/.local/bin/miniconda3/envs/homl1/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


nan

In [1]:
from sklearn.datasets import make_moons

In [5]:
X, y = make_moons(n_samples=10_000, random_state=42)
X.shape, y.shape

((10000, 2), (10000,))

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)
X_train.shape, y_test.shape

((8000, 2), (2000,))

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
        estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
        voting="hard",
)

In [10]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"{clf.__class__.__name__:14s} {accuracy_score(y_test, y_pred):.2f}")

LogisticRegression 0.91
RandomForestClassifier 1.00
SVC            1.00
VotingClassifier 1.00
